In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import random
import datetime

import tensorflow as tf
import numpy as np
import scipy.misc
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data
slim = tf.contrib.slim

In [2]:
mnist = input_data.read_data_sets('MNIST_data')
x_size, y_size = 28, 28
n_classes = 10
default_collection = 'nodes'

def timestamp():
    d = datetime.datetime.now()
    return d.strftime("%Y/%m/%d/%X")

timestamp()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


'2016/12/01/22:12:05'

In [3]:
def build_cnn(inputs, *, n_conv, conv_base, conv_mul, conv_size, pool_size, collection=default_collection):
    l = inputs
    for i in range(n_conv):
        n_filters = conv_base * conv_mul ** i
        l = slim.conv2d(l, n_filters, [conv_size, conv_size],
                        scope='Conv{}'.format(i+1), outputs_collections=collection)
        l = slim.max_pool2d(l, [pool_size, pool_size], scope='MaxPool{}'.format(i+1),
                            outputs_collections=collection)
    l = slim.flatten(l)
    
    l = slim.dropout(l, 0.5, scope='Dropout', outputs_collections=collection)
    l = slim.fully_connected(l, 10, activation_fn=None, scope='Output',
                             outputs_collections=collection)
    return l

def build_loss(logits, y_true):
    logloss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_true))
    return logloss

In [4]:
batch_size = None

n_conv = 2
conv_base = 32
conv_mul = 2
conv_size = 5
pool_size = 2

graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('model') as vs:
        x_ph = tf.placeholder(tf.float32, shape=[batch_size, x_size * y_size])
        x_image = tf.reshape(x_ph, [-1, x_size, y_size, 1])
        y_ph = tf.placeholder(tf.int64, shape=[batch_size])

        logits = build_cnn(x_image, n_conv=n_conv, conv_base=conv_base, conv_mul=conv_mul,
                           conv_size=conv_size, pool_size=pool_size)

        prediction = tf.nn.softmax(logits)

        loss = build_loss(logits, y_ph)

        optimizer = tf.train.AdamOptimizer().minimize(loss)

        correct_prediction = tf.equal(tf.argmax(prediction, 1), y_ph)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    # Code to use of tensorboard
    with tf.name_scope('summaries'):
        tf.scalar_summary('log_loss', loss)
        tf.scalar_summary('acc', accuracy)
        merged_summary = tf.merge_all_summaries()

nodes = graph.get_collection(default_collection)

In [6]:
n_epochs = 1000

train_batch_size = 512

best_acc = 0.0
path = '/tmp/tf/' + timestamp()
with tf.Session(graph=graph) as session:
    saver = tf.train.Saver()
    tf.global_variables_initializer().run()
    print('Initialized')
    train_writer = tf.train.SummaryWriter(
        path+'/train', session.graph)
    val_writer = tf.train.SummaryWriter(
        path+'/val', session.graph)
    
    for epoch in range(n_epochs):
        x_batch, y_batch = mnist.train.next_batch(train_batch_size)
        _, summary, acc, l = session.run([optimizer, merged_summary, accuracy, loss],
                                         feed_dict={x_ph: x_batch, y_ph: y_batch})
        train_writer.add_summary(summary, epoch)
        
        x_batch, y_batch = mnist.validation.next_batch(train_batch_size)
        summary, acc, l = session.run([merged_summary, accuracy, loss],
                                      feed_dict={x_ph: x_batch, y_ph: y_batch})
        val_writer.add_summary(summary, epoch)
        
        if acc > best_acc:
            best_acc = acc
            saver.save(session,"Mnist_NLA.ckpt")
        print("Current Validation accuracy is: {:<4.2%}".format(acc))
    test_acc = accuracy.eval(feed_dict={x_ph: mnist.test.images, y_ph: mnist.test.labels})
    print("The test accuracy is: {:<4.2%}".format(test_acc))

Initialized
Current Validation accuracy is: 17.58%
Current Validation accuracy is: 22.46%
Current Validation accuracy is: 30.08%
Current Validation accuracy is: 45.12%
Current Validation accuracy is: 43.95%
Current Validation accuracy is: 52.93%
Current Validation accuracy is: 56.25%
Current Validation accuracy is: 61.52%
Current Validation accuracy is: 63.67%
Current Validation accuracy is: 60.74%
Current Validation accuracy is: 66.80%
Current Validation accuracy is: 71.48%
Current Validation accuracy is: 70.51%
Current Validation accuracy is: 71.68%
Current Validation accuracy is: 74.80%
Current Validation accuracy is: 70.90%
Current Validation accuracy is: 72.85%
Current Validation accuracy is: 75.59%
Current Validation accuracy is: 73.44%
Current Validation accuracy is: 75.98%
Current Validation accuracy is: 75.39%
Current Validation accuracy is: 78.32%
Current Validation accuracy is: 78.71%
Current Validation accuracy is: 80.08%
Current Validation accuracy is: 83.59%
Current Valid

In [28]:
x = mnist.test.next_batch(1000)

In [38]:
np.all(x[0] == mnist.test.next_batch(1000)[0])

False